In [20]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np
print ('load text documents in shape of lists of comments and lists of labels') 

def cleanup_text(doc_text):
    #doc_text = re.sub("[^a-zA-Z]"," ", doc_text)    
    doc_text = re.sub(" http[.s]*\:.* ", " url ", doc_text)
    return doc_text

# load data
comments = []
labels = []
with open('comments.csv', 'rb') as f:
    for line in f:
        sep_loc = line.rfind(',')
        comments.append(cleanup_text(line[:sep_loc].decode('utf-8')))
        labels.append(int(line[sep_loc+1:]))
print ('there are a total of %d comments in dataset ' % len(comments))

MAX_NB_WORDS = 20000
MAX_SEQUENCE_LENGTH = 100
batch_size = 32

tokenizer = Tokenizer(nb_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(comments)
sequences = tokenizer.texts_to_sequences(comments)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

# random split train and test
perm_idx = np.random.permutation(len(comments))
ntrain = int(0.8*len(comments))
ntest = len(comments) - ntrain
train_idx = perm_idx[:ntrain]
test_idx = perm_idx[ntrain:]
# get train and test sets' X and Y
x_train = sequences[train_idx]
x_test = sequences[test_idx]
y_train = np.asarray(labels[train_idx])
y_test = np.asarray(labels[test_idx])

print('Pad sequences')
x_train = pad_sequences(x_train, maxlen=MAX_SEQUENCE_LENGTHn)
x_test = pad_sequences(x_test, maxlen=MAX_SEQUENCE_LENGTH)

print('Shape of trainset X and Y:', x_train.shape, y_train.shape)
print('Shape of textset X and Y:', x_test.shape, y_test.shape)


load text documents in shape of lists of comments and lists of labels
there are 48448 comments in training set 
there are 12113 comments in testing set 
[u"I think you'll be golden with b/w roshe run. It's not the latest style/fashion forward, but by now they're a classic and neutral (in a good way, i.e. works with lots of outfits), and you know she likes them.  I still see them on lots of very stylish girls here in Copenhagen."]
Found 4 unique tokens.
('Shape of data tensor:', (2L, 100L))
('Shape of label tensor:', (100L,))


In [ ]:
from __future__ import print_function

from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM

print('Build model...')
model = Sequential()
model.add(Embedding(MAX_NB_WORDS, 128))
model.add(LSTM(128, dropout=0.2, dropout=0.2))
model.add(Dense(9, activation='softmax'))

# try using different optimizers and different optimizer configs
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print('Train...')
model.fit(x_train, y_train,
          batch_size=batch_size,
          epoch=15,
          validation_data=(x_test, y_test))
score, acc = model.evaluate(x_test, y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)
